In [49]:
#!pip install backtrader
#!pip install yfinance
#!pip install --upgrade pandas yfinance
!pip install python-dateutil


In [1]:
import backtrader as bt
import pandas as pd
import datetime  # For datetime objects
import os.path  # To manage paths
import sys 
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import yfinance as yf
from dateutil.relativedelta import relativedelta


In [2]:
class CustomCSVData(bt.feeds.GenericCSVData):
    # Définir les lignes (colonnes) du fichier CSV
    lines = ('prob0', 'prob1', 'prob2', 'prob3')

    # Ajouter les paramètres pour le nom de la colonne et le format
    params = (
        ('dtformat', '%Y-%m-%d'),
        ('datetime', 0),
        ('prob0', 1),
        ('prob1', 2),
        ('prob2', 3),
        ('prob3', 4),
        ('time', -1),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
    )


Dans cet exemple, CustomCSVData est configuré pour charger un fichier CSV avec les colonnes de date et de probabilités. Les colonnes time, open, high, low, close, volume, et openinterest sont définies avec l'index -1 car elles ne sont pas présentes dans votre fichier. Vous pouvez accéder à ces données dans votre stratégie en utilisant self.datas[0].prob0, self.datas[0].prob1, etc.


In [19]:
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
    )


    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
        

    def __init__(self):
               
        self.dataclose = self.datas[0].close
        self.dataclose2 = self.datas[1].close
        self.gmm_proba = [self.datas[2].prob0, self.datas[2].prob1,
                          self.datas[2].prob2, self.datas[2].prob3] 
        self.order = None
        self.buy_executed = False
        self.buycomm = None
        self.bar_executed =0
        self.sell_day = None
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

                self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            # Ajouter des logs supplémentaires pour déboguer
            self.log('Order Details: Size: %s, Price: %s, Value: %s' % 
                 (order.size, order.price, order.created.value))
            if order.info:
                self.log('Order Info: %s' % order.info)

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
             

In [20]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    
    cerebro.addstrategy(TestStrategy)
    
    # Create a Data Feed
    data1 = bt.feeds.YahooFinanceCSVData(
        dataname='Bloomberg_Barclays_US_Government_Inflation-Linked_7_to_10_Years.csv',
        # Do not pass values before this date
        fromdate=datetime.datetime(2017, 1, 10),
        # Do not pass values after this date
        todate=datetime.datetime(2023, 12, 31),
        reverse=False)
    
    data2=bt.feeds.YahooFinanceCSVData(
        dataname='SnPClean.csv',
        # Do not pass values before this date
        fromdate=datetime.datetime(2017, 1, 10),
        # Do not pass values after this date
        todate=datetime.datetime(2023, 12, 31),
        reverse=False)
    
    GMM = CustomCSVData(
        dataname='GMM_Proba.csv',
        fromdate=datetime.datetime(2017, 1, 10),
        todate=datetime.datetime(2023, 12, 31),
        nullvalue=0.01,
)
    HMM = CustomCSVData(
        dataname='HMM_Proba.csv',
        fromdate=datetime.datetime(2017, 1, 10),
        todate=datetime.datetime(2023, 12, 31),
        nullvalue=0.01,
)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data1)
    cerebro.adddata(data2)
    cerebro.adddata(GMM)
    cerebro.adddata(HMM)

   
    cerebro.broker.setcash(1000000.0)
    cerebro.broker.setcommission(commission=0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    

Starting Portfolio Value: 1000000.00
2017-01-13, Close, 296.87
2017-01-13, BUY CREATE, 296.87
2017-01-13, Close, 296.87
2017-01-13, Close, 296.87
2017-01-13, Close, 296.87
2017-01-20, BUY EXECUTED, Price: 296.28, Cost: 296.28, Comm 0.00
2017-01-20, Close, 296.28
2017-01-20, SELL CREATE, 296.28
2017-01-20, Close, 296.28
2017-01-20, Close, 296.28
2017-01-20, Close, 296.28
2017-01-20, Close, 296.28
2017-01-27, SELL EXECUTED, Price: 296.89, Cost: 296.28, Comm 0.00
2017-01-27, OPERATION PROFIT, GROSS 0.61, NET 0.61
2017-01-27, Close, 296.89
2017-01-27, Close, 296.89
2017-01-27, Close, 296.89
2017-01-27, Close, 296.89
2017-01-27, Close, 296.89
2017-02-03, Close, 296.38
2017-02-03, Close, 296.38
2017-02-03, Close, 296.38
2017-02-03, Close, 296.38
2017-02-03, Close, 296.38
2017-02-10, Close, 297.79
2017-02-10, Close, 297.79
2017-02-10, Close, 297.79
2017-02-10, Close, 297.79
2017-02-10, Close, 297.79
2017-02-17, Close, 297.24
2017-02-17, Close, 297.24
2017-02-17, Close, 297.24
2017-02-17, Clos

2021-02-05, Close, 371.18
2021-02-05, Close, 371.18
2021-02-05, Close, 371.18
2021-02-05, Close, 371.18
2021-02-12, Close, 371.25
2021-02-12, Close, 371.25
2021-02-12, Close, 371.25
2021-02-12, Close, 371.25
2021-02-19, Close, 364.99
2021-02-19, Close, 364.99
2021-02-19, Close, 364.99
2021-02-19, Close, 364.99
2021-02-19, Close, 364.99
2021-02-26, Close, 363.34
2021-02-26, BUY CREATE, 363.34
2021-02-26, Close, 363.34
2021-02-26, Close, 363.34
2021-02-26, Close, 363.34
2021-02-26, Close, 363.34
2021-03-05, BUY EXECUTED, Price: 361.40, Cost: 361.40, Comm 0.00
2021-03-05, Close, 361.40
2021-03-05, SELL CREATE, 361.40
2021-03-05, Close, 361.40
2021-03-05, Close, 361.40
2021-03-05, Close, 361.40
2021-03-05, Close, 361.40
2021-03-12, SELL EXECUTED, Price: 361.42, Cost: 361.40, Comm 0.00
2021-03-12, OPERATION PROFIT, GROSS 0.02, NET 0.02
2021-03-12, Close, 361.42
2021-03-12, Close, 361.42
2021-03-12, Close, 361.42
2021-03-12, Close, 361.42
2021-03-12, Close, 361.42
2021-03-19, Close, 360.35
2

In [ ]:
cerebro.plot()